## Exercise 2: Rigid and perspective transformations in homogeneous coordinates


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In these exercises we will assume a modern camera with completely square pixels. What are the skew parameters then?

- The skew parameters α and β, where α = 1 and β=0. 

### Mathematical exercises: Pinhole camera

- **Exercise 2.1**. Reuse the `box3` function from last week. Assume that f = 600, α = 1, β = 0, and δx = δy = 400. Given a traditional camera, what is the resolution in pixels?

For this camera the sensor has 2 × 400 = 800 pixels along each dimension i.e. a resolution of **800 × 800 pixels.**

$$ K = \begin{bmatrix} f & \beta f & \delta_x \\ 0 & \alpha f & \delta_y \\ 0 & 0 & 1 \end{bmatrix}$$


$$ K = \begin{bmatrix} 600 & 0 & 400 \\ 0 & 600 & 400 \\ 0 & 0 & 1 \end{bmatrix}$$

Also assume $R = I$, and $t = [0,.2,1.5]^T.$ Use `projectpoint` from last week, to project the points.

In [2]:
def box3d(n):
    comb=[(0.5,0.5),(0.5,-0.5),(-0.5,0.5),(-0.5,-0.5)]
    a=np.linspace(-0.5, 0.5, n)
    box=[0,0,0]

    for j in comb:
        for i in a:
            p=[i,j[0],j[1]]
            box=np.vstack([box, p])

    for j in comb:
        for i in a[1:-1]:
            p=[j[0],i,j[1]]
            box=np.vstack([box, p])

    for j in comb:
        for i in a[1:-1]:
            p=[j[0],j[1],i]
            box=np.vstack([box, p])

    for i in a:
        cross1=[i,0,0]   
        box=np.vstack([box, cross1])
    for i in a:
        cross2=[0,i,0]   
        box=np.vstack([box, cross2])
    for i in a:
        cross3=[0,0,i]   
        box=np.vstack([box, cross3])
      
    box=box[1:]

    return box


def projectpoints(K,Cam,Q):
    #projection matrix P
    P=K@Cam
    
    [m,n]=np.shape(Q)
    cnt=np.ones(m)
    Q_ext=np.column_stack((Q,cnt))
    
    Q_ext=np.transpose(Q_ext)
    
    projection=P@(Q_ext)
    
    qx=np.transpose([projection[0]/projection[2]])
    qy=np.transpose([projection[1]/projection[2]])
    
    project2D=np.column_stack((qx,qy))
    
    project3D=np.transpose(projection)
    
    return P, project2D, project3D

In [3]:
K=np.array([[600,0,400],[0,600,400],[0,0,1]]) #camera matrix
R=np.array([[1,0,0],[0,1,0],[0,0,1]]) #rotation
t=np.transpose([0,0.2,1.5]) #translation
Q=box3d(16) #n points in 3D to be projected into the camera

Cam=np.column_stack((R,t))

P,points2D,points3D=projectpoints(K,Cam,Q)
print('The projection matrix of the camera is:')
print(P)

The projection matrix of the camera is:
[[600.    0.  400.  600. ]
 [  0.  600.  400.  720. ]
 [  0.    0.    1.    1.5]]


Are all the points are captured by the image sensor? 


In [4]:
list_coord=[]
for coord in points2D: 
    if (coord[0]> 800) or (coord[1] > 800):
        list_coord.append(list(coord))

print('The points not caputed by the image sensor are:',list_coord)

The points not caputed by the image sensor are: [[100.0, 820.0], [140.0, 820.0], [180.0, 820.0], [220.0, 820.0], [260.0, 820.0], [300.0, 820.0], [340.0, 820.0], [380.0, 820.0], [420.0, 820.0], [460.0, 820.0], [500.0, 820.0], [540.0, 820.0], [580.0, 820.0], [620.0, 820.0], [660.0, 820.0], [700.0, 820.0]]


In [ ]:
p1 = 

<font color='darkblue'> Some points have an y value greater than 800, and are not visible in the image, as they are outside the image sensor.
 

Where does the corner **$P_1 = [−0.5, −0.5, −0.5]$** project to?

In [5]:
#Function created to find a specific 2D projected point. 
def find_projected2Dpoint(Q,points2D,P1):
    for ind,row in enumerate(Q):    
        if np.all(row==P1):
            coord=points2D[ind]
    return coord


coord= find_projected2Dpoint(Q,points2D,P1)

print('The corner P1 project to:',coord)

NameError: name 'P1' is not defined

- **Exercise 2.2**. Create a new or change your function `projectpoint` to a version that also takes `dist` as an input. The list dist should contain the distortion coefficients $[k3, k5, k7, . . . ].$ Make the function work for at least 3 coefficients.


In [14]:
def projectpoints_2D(K,Cam,Q):
    
    [m,n]=np.shape(Q)
    cnt=np.ones(m)
    Q_ext=np.column_stack((Q,cnt))
    
    Q_ext=np.transpose(Q_ext)
    
    projection=Cam@(Q_ext)
    
    Pz=projection[2]
    qx=np.transpose([projection[0]/projection[2]])
    qy=np.transpose([projection[1]/projection[2]])
    points2D=np.column_stack((qx,qy))
    r=np.sqrt(points2D.T[0]**2+points2D.T[1]**2)
    dist= points2D.T*(1-0.2*(r**2))
    
    Px= dist[0]*Pz
    Py= dist[1]*Pz
    points3D=np.column_stack((Px,Py,Pz))
    
    P=K@points3D.T
    
    qx_f=np.transpose([P[0]/P[2]])
    qy_f=np.transpose([P[1]/P[2]])
    
    dist_points=np.column_stack((qx_f,qy_f))
    
    return dist_points

Where does the corner **P1** project to?

In [15]:
dist_points=projectpoints_2D(K,Cam,Q)
print(r)

coord= find_projected2Dpoint(Q,dist_points,P1)

print('The corner P1 project to:',coord)

[[-0.25      ]
 [-0.21666667]
 [-0.18333333]
 [-0.15      ]
 [-0.11666667]
 [-0.08333333]
 [-0.05      ]
 [-0.01666667]
 [ 0.01666667]
 [ 0.05      ]
 [ 0.08333333]
 [ 0.11666667]
 [ 0.15      ]
 [ 0.18333333]
 [ 0.21666667]
 [ 0.25      ]
 [-0.5       ]
 [-0.43333333]
 [-0.36666667]
 [-0.3       ]
 [-0.23333333]
 [-0.16666667]
 [-0.1       ]
 [-0.03333333]
 [ 0.03333333]
 [ 0.1       ]
 [ 0.16666667]
 [ 0.23333333]
 [ 0.3       ]
 [ 0.36666667]
 [ 0.43333333]
 [ 0.5       ]
 [-0.25      ]
 [-0.21666667]
 [-0.18333333]
 [-0.15      ]
 [-0.11666667]
 [-0.08333333]
 [-0.05      ]
 [-0.01666667]
 [ 0.01666667]
 [ 0.05      ]
 [ 0.08333333]
 [ 0.11666667]
 [ 0.15      ]
 [ 0.18333333]
 [ 0.21666667]
 [ 0.25      ]
 [-0.5       ]
 [-0.43333333]
 [-0.36666667]
 [-0.3       ]
 [-0.23333333]
 [-0.16666667]
 [-0.1       ]
 [-0.03333333]
 [ 0.03333333]
 [ 0.1       ]
 [ 0.16666667]
 [ 0.23333333]
 [ 0.3       ]
 [ 0.36666667]
 [ 0.43333333]
 [ 0.5       ]
 [ 0.25      ]
 [ 0.25      ]
 [ 0.25   

NameError: name 'P1' is not defined

Are all the points captured by the image sensor?

In [154]:
list_coord=[]
for coord in dist_points:
    if (coord[0]> 800) or (coord[1] > 800):
        list_coord.append(list(coord))

print('The points not caputed by the image sensor are:',list_coord)

The points not caputed by the image sensor are: []


<font color='darkblue'> All the points are now projecting inside the image, and will thus be visible.

Plot the results and try changing the distortion coefficients. Do they behave as they should?